# Python Data, 2024

---

* [Spojování dat](#Spojov%C3%A1n%C3%AD-dat),
    - [spojování s CONCAT](#Spojování-s-CONCAT),
    - [spojování s CONCAT a JOIN](#concat-a-volitelný-argument-join),
    - [spojování s metodou APPEND](#Spojování-pomocí-metody-APPEND),
    - [spojování s MERGE](#Spojování-pomocí-merge),
    - [spojování s metodou JOIN](#Spojení-pomocí-indexů,-join),
* [Agregace](#Agregace),
    - [jednoduchá agregace](#Jednoduchá-agregace),
    - [seskupování GROUPBY](#Seskupování-groupby),
    - [agregace](#Agregace-s-metodami),
    - [filtrování](#Filtrování),
    - [transformace](#Transformace),
    - [apply](#Metoda-apply),
* [Pivot tabulka](#Pivot-tabulky),
    - [úvodní motivace](),
    - [syntaxe tabulky](),
    - [doplňující možnosti](),
    - [cvičení 3]().



<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.APOjuHGvkM0wQaUx9ELKAgHaHa%26pid%3DApi&f=1&ipt=a8b2b692c40e190d4b28c164bf13e6c944702a69944035a4ef9053aa6ea1b190&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

<br>

## Spojování dat

---

Takovými operacemi si můžeš představit jednoduché operace jako **spojování** (*konkatenace*) **dvou a více datasetů** až po složitější *JOINY* podobně jako u databází.

`pandas` obsahuje různé *funkce* a *metody*, které umožňují tento způsob práce.

* funkce `concat`,
* funkce `merge`,
* metoda `append` (outdated),
* metoda `join`.

### Spojování s CONCAT

---

Funkce `concat()` v `pandas` umožňuje spojit **dva nebo více**:
1. **sloupečků** tabulky,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,
3. **tabulek** samotných,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,

Použití funkce `concat()` spočívá **v předání seznamu objektů**, které chceš spojit, a parametru `axis`, který určuje osu, podle které se májí objekty spojit:

#### Spojení sloupečků, CONCAT

---

In [1]:
from pandas import concat, Series, DataFrame

In [2]:
sloupec_1 = Series(['A', 'B', 'C'], index=[1, 2, 3])
sloupec_2 = Series(['D', 'E', 'F'], index=[4, 5, 6])

In [3]:
sloupec_1

1    A
2    B
3    C
dtype: object

In [4]:
sloupec_2

4    D
5    E
6    F
dtype: object

In [5]:
spojene_sloupce = concat([sloupec_1, sloupec_2])

<br>

Spojení **horizontální** (pod sebou):

In [6]:
horizontalni_spoj = concat([sloupec_1, sloupec_2], axis=1)

In [7]:
horizontalni_spoj

,0,1
1,A,NaN
2,B,NaN
3,C,NaN
4,NaN,D
5,NaN,E
6,NaN,F


<br>

Spojení **vertikální** (vedle sebe):

In [8]:
vertikalni_spoj = concat([sloupec_1, sloupec_2], axis=0)

In [9]:
vertikalni_spoj

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

<br>

#### Spojení tabulek, CONCAT

---

In [12]:
uzivatele_tab_1 = {'jmeno': ['Matouš', 'Marek', 'Lukáš'],
               'vek': [25, 30, 35]}

In [13]:
uzivatele_tab_2 = {'jmeno': ['Petr', 'Jan', 'Michal'],
                   'vek': [40, 45, 50]}

In [14]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [15]:
uzivatele_tab_1_df

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35


In [16]:
uzivatele_tab_2_df

,jmeno,vek
0,Petr,40
1,Jan,45
2,Michal,50



#### Samotné spojení dvou tabulek

---

In [17]:
df_spojene_tab = concat([uzivatele_tab_1_df, uzivatele_tab_2_df], axis=0)

In [18]:
df_spojene_tab

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35
0,Petr,40
1,Jan,45
2,Michal,50


#### Co s duplicitními Indexy?

---

Tady se ovšem **zduplikovali hodnoty** některých indexů.

Ty je potřeba opravit tímto postupem:
1. **Vytvořím nový sloupeček** pro indexy,
2. **odstraním starý sloupeček** s duplicitami.

In [19]:
df_spojene_tab = df_spojene_tab.reset_index()  # Přidá nový Index

In [20]:
df_spojene_tab

,index,jmeno,vek
0,0,Matouš,25
1,1,Marek,30
2,2,Lukáš,35
3,0,Petr,40
4,1,Jan,45
5,2,Michal,50


In [22]:
df_spojene_tab = df_spojene_tab.drop("index", axis=1)

<br>

Kontrola nového rozsahu **pro Index**:

In [24]:
df_spojene_tab.index

RangeIndex(start=0, stop=6, step=1)

<br>

Pozůstatky sloupečků:

In [25]:
df_spojene_tab.columns

Index(['jmeno', 'vek'], dtype='object')

In [26]:
df_spojene_tab

,jmeno,vek
0,Matouš,25
1,Marek,30
2,Lukáš,35
3,Petr,40
4,Jan,45
5,Michal,50


<br>

Nastavení nových Indexů **podle jmen**:

In [31]:
df_spojene_tab.index.names?

Type:        property
String form: <property object at 0x7f0eb75c7c70>
Docstring:   <no docstring>

<br>

Doplníš jméno **pro Index**:

In [33]:
df_spojene_tab.index.names = ["Index ze jmén"]

In [34]:
df_spojene_tab.index.names

FrozenList(['Index ze jmén'])

In [35]:
df_spojene_tab

,jmeno,vek
Index ze jmén,,
0,Matouš,25
1,Marek,30
2,Lukáš,35
3,Petr,40
4,Jan,45
5,Michal,50


Pro odchytávání **duplicitních indexů** můžeš doplnit parametr `verify_integrity=True`, případně pokud je irelevantní, ignorovat jej úplně `ignore_index=True`.

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí CONCAT**

In [1]:
data_populace = {
    "City": ["New York", "London", "Tokyo"],
    "Population": [8.4, 9.0, 13.9],
}

data_plocha = {
    "City": ["New York", "London", "Tokyo"],
    "Area": [783, 1572, 2194],
}

data_kontinent = {
    "City": ["New York", "London", "Tokyo"],
    "Continent": ["North America", "Europe", "Asia"],
}

# Vytvoř funkci "spoj_data_mest", která spojí 3 předchozí DF

<details>
    <summary>▶️ Řešení</summary>
    
```python
from pandas import DataFrame, concat

data_populace = {
    "City": ["New York", "London", "Tokyo"],
    "Population": [8.4, 9.0, 13.9],
}

data_plocha = {
    "City": ["New York", "London", "Tokyo"],
    "Area": [783, 1572, 2194],
}

data_kontinent = {
    "City": ["New York", "London", "Tokyo"],
    "Continent": ["North America", "Europe", "Asia"],
}

populace_df = DataFrame(data_populace)
plocha_df = DataFrame(data_plocha)
kontinent_df = DataFrame(data_kontinent)


def spoj_data_mest(populace, plocha, kontinent):
    """
    Spojí tři tabulky horizontálně podle sloupce City.
    """
    return concat([populace.set_index("City"),
                      plocha.set_index("City"),
                      kontinent.set_index("City")], axis=1).reset_index()


result = spoj_data_mest(populace_df, plocha_df, kontinent_df)
print(result)
```
</details>

<img src="https://imgs.search.brave.com/wR9sz6BPwR_LCDm6Esd_B9VgUBSbW4wbTPi2T9DrdBc/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQwLzM1LzIx/LzM2MF9GXzg0MDM1/MjE1Ml84N1N2T0kx/REM2RElCaHkxc0xz/VnpnYmNIUk1UZno0/cC5qcGc" width="230" style="margin-left:auto; margin-right:auto"/>

### CONCAT a volitelný argument JOIN

---

V jednoduchých ukázkách, jako jsou ty výše, stačilo **tabulky a sloupečky spojit**.

To prakticky není vždy ideální řešení, protože některé sloupečky můžou, **ale nemusí být shodné**.

In [36]:
from pandas import DataFrame, concat

In [37]:
data_tab_1_df = DataFrame(
    {"A": ["A1", "A2"], "B": ["B1", "B2"], "C": ["C1", "C2"]},
    index=[1, 2]
)

In [38]:
data_tab_2_df = DataFrame(
    {"B": ["B3", "B4"], "C": ["C3", "C4"], "D": ["D3", "D4"]},
    index=[3, 4]
)

In [39]:
data_tab_1_df

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [40]:
data_tab_2_df

,B,C,D
3,B3,C3,D3
4,B4,C4,D4



#### Horizontální spojení dvou tabulek

---

In [41]:
vystup_tab_df = concat([data_tab_1_df, data_tab_2_df])  # axis=0

In [43]:
vystup_tab_df

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


Pokud některá data chybějí, jsou automaticky vyplněná **neznámými hodnotami**.

Řešením takové situace můžeš být zavedení jiného **způsobu spojování**.

Tedy nepoužívat defaultní argument `join='outer'`, ale `join='inner'`:

In [51]:
vystup_bez_nan_df = concat([data_tab_1_df, data_tab_2_df],
                           join='inner',
                           axis=0)

In [52]:
vystup_bez_nan_df

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


<img src="https://imgs.search.brave.com/IZ8jkD1GaDV2tXwgwzM1Xawk-jBMejygby1KcermQp8/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQ2LzQ3LzE1/LzM2MF9GXzg0NjQ3/MTU2MF95Y281RnhM/UG5RR1c4NW1OWHBD/UmcxUWptSzRiMnp3/eS5qcGc" width="200" style="margin-left:auto; margin-right:auto"/>



### Spojování pomocí `MERGE`

---

Další funkcí pro spojování `DataFrame` objektů je `merge`.

Tato funkce je vhodná pro spojování DataFrame objektů, které **mají společné sloupce**.

Můžeš lépe zadávat typ spojení (parametr `how='inner' | 'outer' | 'left' | 'join'`).

Dále ti umožní **definovat sloupec, nebo sloupce**, na kterých chceš spojení provést (parametr `on`).

In [53]:
from pandas import merge

In [55]:
uzivatele_tab_1 = {
    'jmeno': ['Alice', 'Bob', 'Charlie', 'David'],
    'vek': [25, 30, 35, 40],
    'mesto': ['Brno', 'Praha', 'Plzen', 'Ostrava']
}

In [56]:
uzivatele_tab_2 = {
    'jmeno': ['Alice', 'David', 'Emma', 'Frank'],
    'pocet_prijemcu': [100, 200, 150, 250]
}

In [57]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [58]:
uzivatele_tab_1_df

,jmeno,vek,mesto
0,Alice,25,Brno
1,Bob,30,Praha
2,Charlie,35,Plzen
3,David,40,Ostrava


In [59]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


#### Spojení FULL JOIN

---

In [60]:
mergnute_df = merge(uzivatele_tab_1_df, uzivatele_tab_2_df,
                    on='jmeno',
                    how='outer')

In [61]:
mergnute_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25.0,Brno,100.0
1,Bob,30.0,Praha,NaN
2,Charlie,35.0,Plzen,NaN
3,David,40.0,Ostrava,200.0
4,Emma,NaN,NaN,150.0
5,Frank,NaN,NaN,250.0


#### Spojené INNER MERGE

---

In [62]:
mergnute_inner_df = merge(uzivatele_tab_1_df,
                          uzivatele_tab_1_df,
                          on='jmeno',
                          how='inner')

In [63]:
mergnute_inner_df

,jmeno,vek_x,mesto_x,vek_y,mesto_y
0,Alice,25,Brno,25,Brno
1,Bob,30,Praha,30,Praha
2,Charlie,35,Plzen,35,Plzen
3,David,40,Ostrava,40,Ostrava


Funkce sama **doplní přípony**, aby rozlišila **mezi oběma původními sloupci**.

**Pokud potřebuješ vlastní přípony**, můžeš vyzkoušet volitelný argument pro `suffixes`:

In [73]:
df8 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [1, 2, 3, 4]
})

In [74]:
df9 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [3, 1, 4, 2]
})

In [75]:
df8

,jmeno,poradi
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [76]:
df9

,jmeno,poradi
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [77]:
vystup_konflikt = merge(df8, df9, on="jmeno")

In [78]:
vystup_konflikt

,jmeno,poradi_x,poradi_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [79]:
vystup_vlastni_pripony = merge(df8, df9, on='jmeno', suffixes=('_DF8', '_DF9'))

In [80]:
vystup_vlastni_pripony

,jmeno,poradi_DF8,poradi_DF9
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Obecně platí, že pokud potřebuješ *spojit* dva nebo více `DataFrame` objektů podle **společného sloupce nebo sloupců**, použij funkci `merge()`.

Pokud chceš jenom **přidat další řádky nebo sloupce** do existujícího `DataFrame` objektu, použij funkci `concat`.

In [64]:
uzivatele_tab_1_df

,jmeno,vek,mesto
0,Alice,25,Brno
1,Bob,30,Praha
2,Charlie,35,Plzen
3,David,40,Ostrava


In [65]:
uzivatele_tab_2_df

,jmeno,pocet_prijemcu
0,Alice,100
1,David,200
2,Emma,150
3,Frank,250


In [67]:
vystup_left_join_df = merge(uzivatele_tab_1_df,
                         uzivatele_tab_2_df,
                         on='jmeno',
                         how='left')

In [68]:
vystup_left_join_df

,jmeno,vek,mesto,pocet_prijemcu
0,Alice,25,Brno,100.0
1,Bob,30,Praha,NaN
2,Charlie,35,Plzen,NaN
3,David,40,Ostrava,200.0


V ukázce výš je použitý *left join*.

Tedy ve výsledku uvidíš **celou první** (levou tabulku) a z druhé pouze ty záznamy, které mají ve spojovacím sloupci `jmeno` společnou hodnotu.

### Spojení pomocí indexů, `join`

---

Tato metoda slouží k propojení dvou DataFrame objektů na základě **jejich indexů nebo hodnot**.

Je velice podobná funkci `merge` ale je přímo součástí `DataFrame` objektu a je snazší ji aplikovat:

In [ ]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
     'vek': [25, 30, 35]
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
    'pocet_prijemcu': [100, 150, 200]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
df_uzivatele_2.index.names

In [ ]:
df_uzivatele_1 = df_uzivatele_1.set_index('jmeno')

In [ ]:
df_uzivatele_2 = df_uzivatele_2.set_index('jmeno')

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
vysledek_join_metody = df_uzivatele_1.join(df_uzivatele_2)

In [ ]:
vysledek_join_metody

Hlavní rozdíl mezi těmito funkcemi je způsob určení **sloupce nebo sloupců**, podle kterých se má propojení provést.

1. Metoda `join()` propojuje DataFrame objekty **na základě jejich indexů**.
2. Funkce `merge()` umožňuje propojit `DataFrame` objekty **na základě hodnoty v jednom nebo více sloupcích**.

### Souhrn ke spojování

---

| Objekt | Kdy jej použít? | Směr spojení |
| :- | :- | :- |
| `concat` | Když chceš poskládat tabulky na sebe (řádky) nebo vedle sebe (sloupce). | Vertikálně / Horizontálně | 
| `merge` |	Když chcete spojit tabulky podle společného klíče/sloupce, podobně jako SQL JOIN. | Horizontálně |
| `join` | Když chcete spojit tabulky podle společného indexu (ne klíče/sloupce). | Horizontálně |
| `append` | (Zastaralé) Používalo se k přidání řádků, nyní používejte concat. | Vertikálně |

<br>

**🧠 CVIČENÍ 🧠, procvič si spojování**

Tvým úkolem je propojit tyto tabulky podle zadání:
1. Použij spojovací funkci pro spojení objektů typu `DataFrame` , které vytvoříš z proměnných `data_zamestnanci_1` a `data_zamestnanci_2`,
2. použij spojovací funkci, které zadáš vzniklý objekt typu `DataFrame`  z bodu 1. a druhý `DataFrame`, který vytvoříš z proměnné `data_oddeleni` (spoj přes sloupeček `"oddělení_id"`),
3. vytvoř výsledný objekt typu `DataFrame` , který obsahuje pouze zaměstnance z oddělení `IT`.

In [ ]:
data_zamestnanci_1 = {
    'id': [1, 2, 3],
    'jméno': ['Jan', 'Marie', 'Petr'],
    'příjmení': ['Novák', 'Svobodová', 'Dvořák'],
    'oddělení_id': [100, 200, 100]
}

data_zamestnanci_2 = {
    'id': [4, 5],
    'jméno': ['Anna', 'Josef'],
    'příjmení': ['Kopecká', 'Vondráček'],
    'oddělení_id': [200, 100]
}

data_oddeleni = {
    'oddělení_id': [100, 200],
    'název_oddělení': ['IT', 'HR']
}

<details>
    <summary>▶️ Řešení</summary>
    
```python
df_zamestnanci_1 = pd.DataFrame(data_zamestnanci_1)
df_zamestnanci_2 = pd.DataFrame(data_zamestnanci_2)
df_oddeleni = pd.DataFrame(data_oddeleni)

df_vsichni_zamestnanci = pd.concat([df_zamestnanci_1, df_zamestnanci_2], ignore_index=True)

# df1 = pd.concat(
#     [pd.DataFrame(data) for data in [data_zamestnanci_1, data_zamestnanci_2]]
# ).reset_index(drop = True).set_index('id')

df_vsechna_oddeleni = pd.merge(df_vsichni_zamestnanci, df_oddeleni, on="oddělení_id")

df_jen_it = df_vsechna_oddeleni[df_vsechna_oddeleni["název_oddělení"] == "IT"]
```
</details>

## Agregace

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.HtBtNx6dodlNptXV1QGHWQHaG0%26pid%3DApi&f=1&ipt=3a4d1dfa14a495127b4ece5cf689f00ed0c9e0836853358412e18161ae7c505f&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>


*Seskupování* neboli **agregace** jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

---

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [2]:
from pandas import read_csv

In [3]:
df_nemovitosti = read_csv("../onsite/housing.csv")

In [4]:
df_nemovitosti.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade
0,1,221900.0,3,1.00,1180,5650,1,3,7
1,2,538000.0,3,2.25,2570,7242,2,3,7
2,3,180000.0,2,1.00,770,10000,1,3,6
3,4,604000.0,4,3.00,1960,5000,1,5,7
4,5,510000.0,3,2.00,1680,8080,1,3,8


In [13]:
df_nemovitosti.loc[:, "price"].mean()

438075.0

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

### Seskupování GROUPBY

---

Metoda `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [14]:
from pandas import DataFrame

In [15]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'číselná hodnota': range(6)},
)

In [16]:
df_pokus_s_cisly

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


#### Klasické seskupení podle sloupečku

---

In [17]:
df_pokus_s_cisly.groupby("klíč")

In [18]:
type(df_pokus_s_cisly.groupby("klíč"))

pandas.core.groupby.generic.DataFrameGroupBy

Metoda standardně vrací `DataFrameGroupBy`.

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [19]:
df_pokus_s_cisly.groupby("klíč").sum()

,číselná hodnota
klíč,
A,3
B,5
C,7


Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

<br>

#### Sloupečkové označování

---

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [20]:
df_pokus_s_cisly

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [21]:
df_pokus_s_cisly.groupby("klíč")["číselná hodnota"]

In [22]:
df_pokus_s_cisly.groupby("klíč")["číselná hodnota"].median()

klíč
A    1.5
B    2.5
C    3.5
Name: číselná hodnota, dtype: float64

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `median`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [23]:
for (klic, hodnota) in df_pokus_s_cisly.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota.shape}")

Klic: A; Hodnota=(2, 2)
Klic: B; Hodnota=(2, 2)
Klic: C; Hodnota=(2, 2)


<br>

### Agregace s metodami

---

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

#### Metoda agregate

---

In [24]:
import numpy

In [25]:
rng = numpy.random.RandomState(0)

In [26]:
from pandas import DataFrame

In [27]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'data_1': range(6),
        'data_2': rng.randint(0, 10, 6)
    }
)

In [28]:
df_pokus_s_cisly

,klíč,data_1,data_2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


Statistické metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [29]:
df_pokus_s_cisly.groupby("klíč").aggregate("sum")

,data_1,data_2
klíč,,
A,3,8
B,5,7
C,7,12


In [30]:
df_pokus_s_cisly.groupby("klíč").aggregate(numpy.median)

,data_1,data_2
klíč,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


In [31]:
df_pokus_s_cisly.groupby("klíč").aggregate(["min", numpy.median, max])

data_1            data_2           
        min median max    min median max
klíč                                    
A         0    1.5   3      3    4.0   5
B         1    2.5   4      0    3.5   7
C         2    3.5   5      3    6.0   9

In [32]:
df_pokus_s_cisly

,klíč,data_1,data_2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
df_ovoce = DataFrame(ovoce_data)

In [ ]:
df_ovoce.head(6)

In [ ]:
vystup = df_ovoce.groupby('pobocka').aggregate({'prodane_mnozstvi': sum})

In [ ]:
vystup = df_ovoce.groupby('pobocka').aggregate({'prodane_mnozstvi': 'sum', 'cena': 'mean'})

In [ ]:
vystup

1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Filtrování

---

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání **seskupených dat** podle splnění **určité podmínky**.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [34]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [35]:
df_ovoce = DataFrame(ovoce_data)

In [37]:
df_ovoce

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [38]:
def vyber_pouze_pobocky_s_prodejem_nad_limit(data, limit: int = 24) -> bool:
    return data['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_pouze_pobocky_s_prodejem_nad_limit`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [39]:
vysledek = df_ovoce.groupby('pobocka').filter(vyber_pouze_pobocky_s_prodejem_nad_limit)

In [40]:
vysledek

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci (`vyber_pouze_pobocky_s_prodejem_nad_limit`),
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

### Transformace

---

Zatímco předchozí výsledky *agregace* pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [41]:
df_ovoce

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [42]:
def vrat_procenta_z_celkoveho_prodeje(udaj) -> float:
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `procento_z_celkoveho_prodeje`:

In [43]:
df_ovoce['procento_z_celkoveho_prodeje'] = df_ovoce.groupby('pobocka')['prodane_mnozstvi'] \
    .transform(vrat_procenta_z_celkoveho_prodeje)

In [44]:
df_ovoce

,pobocka,ovoce,prodane_mnozstvi,cena,procento_z_celkoveho_prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `procento_z_celkoveho_prodeje`.

### Metoda `apply`

---

Metoda `apply` ti také dovolí, používat uživatelem definované funkce na seskupená data.

Následně vrací objekt knihovny `pandas` (buď `DataFrame`, nebo `Series`, a nebo skalární hodnota).

In [45]:
df_ovoce

,pobocka,ovoce,prodane_mnozstvi,cena,procento_z_celkoveho_prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


In [46]:
def vypocitej_vydelek_za_artikl(seskupene):
    seskupene['vydelek_pobocky'] = seskupene['prodane_mnozstvi'] * seskupene['cena']
    return seskupene

<br>

Metoda `apply` je **obecnější a flexibilnější** než `transform`.

`apply` umožňuje použít uživatelem definovanou funkci na každou skupinu po seskupení dat pomocí `groupby`.

Výsledek metody `apply` může mít jiný tvar než původní data.

In [51]:
vystup = df_ovoce.groupby('pobocka', group_keys=False).apply(vypocitej_vydelek_za_artikl)

In [52]:
vystup

,pobocka,ovoce,prodane_mnozstvi,cena,procento_z_celkoveho_prodeje,vydelek_pobocky
0,A,jablko,10,20,40.0,200
1,A,banan,15,12,60.0,180
2,B,jablko,8,22,21.1,176
3,B,banan,30,10,78.9,300
4,C,jablko,20,18,44.4,360
5,C,banan,25,15,55.6,375


In [ ]:
df_ovoce.groupby?

Od posledních verzí frameworku (`1.5.0` a vyšší) platí, že pokud bude výsledkem `DataFrame` nebo `Series` musíš uvést argument pro `group_keys=True`).

<br>

In [53]:
vystup = df_ovoce.groupby('pobocka', group_keys=True).apply(vypocitej_vydelek_za_artikl)

In [54]:
vystup

pobocka   ovoce  prodane_mnozstvi  cena  \
pobocka                                             
A       0       A  jablko                10    20   
        1       A   banan                15    12   
B       2       B  jablko                 8    22   
        3       B   banan                30    10   
C       4       C  jablko                20    18   
        5       C   banan                25    15   

           procento_z_celkoveho_prodeje  vydelek_pobocky  
pobocka                                                   
A       0                          40.0              200  
        1                          60.0              180  
B       2                          21.1              176  
        3                          78.9              300  
C       4                          44.4              360  
        5                          55.6              375

Na první pohled vypadají metody `apply` a `transform` docela podobně.

Metoda `transform` je trochu omezenější než `apply`.

Slouží k aplikaci uživatelem definované nebo vestavěné funkce **na každý prvek** skupiny po seskupení s `groupby`.

`transform` musí vracet hodnotu stejného tvaru jako vstupní data.

Výsledek metody `transform` **má stejný tvar jako původní data**.

#### Apply
* potřebuješ výsledek, který **má jiný tvar než původní data**,
* umí zpracovat **více sloupečků současně**.

#### Transform
* potřebuješ výsledek, který **má stejný tvar jako původní data**,
* umí zpracovat **pouze jeden sloupeček**.

In [55]:
df_rozdily = DataFrame({
    'KLIC': ['A','B','C'] * 3,
    'A': numpy.arange(9),
    'B': [1,2,3] * 3,
})

In [56]:
df_rozdily

,KLIC,A,B
0,A,0,1
1,B,1,2
2,C,2,3
3,A,3,1
4,B,4,2
5,C,5,3
6,A,6,1
7,B,7,2
8,C,8,3


#### `transform` vrací výsledky ve stejném tvaru

---

In [57]:
def vypocitej_sumu(data):
    return data.sum()

In [58]:
seskup_df_rozdily_apply = df_rozdily.groupby('KLIC')['A'].apply(vypocitej_sumu)

In [59]:
seskup_df_rozdily_apply

KLIC
A     9
B    12
C    15
Name: A, dtype: int64

In [60]:
seskup_df_rozdily_trans = df_rozdily.groupby('KLIC')['A'].transform(vypocitej_sumu)

In [61]:
seskup_df_rozdily_trans

0     9
1    12
2    15
3     9
4    12
5    15
6     9
7    12
8    15
Name: A, dtype: int64

#### `apply` umí pracovat s více sloupečky, `transform` jen s jedním

---

In [62]:
df_rozdily

,KLIC,A,B
0,A,0,1
1,B,1,2
2,C,2,3
3,A,3,1
4,B,4,2
5,C,5,3
6,A,6,1
7,B,7,2
8,C,8,3


In [63]:
def vypocitej_rozdil(data):
    return data['B'] - data['A']

In [64]:
df_rozdily.groupby('KLIC').apply(vypocitej_rozdil)

KLIC   
A     0    1
      3   -2
      6   -5
B     1    1
      4   -2
      7   -5
C     2    1
      5   -2
      8   -5
dtype: int64

In [66]:
# df_rozdily.groupby('KLIC').transform(vypocitej_rozdil)

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí GroupBy a agregační funkce**

Máš zadaný takový datový set.

Následně:
1. Pomocí metody `filter` vyber prodejny, které prodaly **alespoň 30 produktů**,
2. na filtrovaném datasetu použijte metodu `apply` pro výpočet **celkového příjmu z prodeje pro každý obchod**.

In [ ]:
from pandas import DataFrame

df_prodej_hardware = DataFrame({
    'prodejna_id': (5, 4, 1, 5, 5, 1, 4, 2, 5, 1, 3, 1, 3, 4, 2, 1, 5, 4, 1, 5),
    'transakce_id': (1278, 1216, 1866, 1872, 1797, 1272, 1880, 1061, 1595, 1879, 1728,
       1341, 1396, 1698, 1018, 1176, 1611, 1395, 1444, 1232),
    'predmet_prodeje': ('grafická_karta', 'SSD', 'RAM', 'procesor', 'grafická_karta',
       'základní_deska', 'SSD', 'SSD', 'grafická_karta', 'RAM',
       'grafická_karta', 'procesor', 'grafická_karta', 'SSD',
       'grafická_karta', 'RAM', 'základní_deska', 'HDD', 'grafická_karta',
       'RAM'),
    'pocet_prodanych_ks': (1,  5,  6,  6,  3,  7,  9, 10,  8,  6,  8,  5,  8, 10,  4, 10,  8,
       10,  2,  5),
    'cena_predmetu': (19500.69874949, 19731.10951735, 14114.15342339, 10953.87914371,
        6535.78851758, 16369.00288429, 13852.2578648 ,  3671.03031723,
       18263.08009763, 16539.476237  , 19021.09830919, 14651.53041357,
       12461.59632075,  8655.73920767, 18688.2054254 , 17388.24584526,
        1381.76406707,  1014.1560027 ,  7841.03565412, 16305.78995025)
})

<details>
    <summary>▶️ Řešení</summary>
    
```python
def vyber_prodej_vetsi_nez_limit(data, limit: int = 30):
    return data['pocet_prodanych_ks'].sum() > limit
    
def vypocitej_celkovy_vydelek_prodejny(skupina):
    return (skupina['pocet_prodanych_ks'] * skupina['cena_predmetu']).sum()

df_filtr_hardware = df_prodej_hardware.groupby('prodejna_id') \
                        .filter(vyber_prodej_vetsi_nez_limit)

#         cilove_prodejny = df_prodej_hardware.groupby('prodejna_id') \
# .filter(lambda prodejna: prodejna.pocet_prodanych_ks.sum() >= 30)


celkovy_vydelek = df_filtr_hardware.groupby('prodejna_id') \
                        .apply(vypocitej_celkovy_vydelek_prodejny)

# prodeje_prodejen = cilove_prodejny.groupby('prodejna_id') \
# .aggregate({'pocet_prodanych_ks': sum})

# df_prodej_hardware["prodejna_id"].unique()

# prijmy_prodejen = pd.DataFrame(df_prodej_hardware.groupby('prodejna_id')['cena_predmetu'] \
#                 .apply(lambda prodejna: prodejna.sum())) \
#                 .rename(columns = { 'cena_predmetu': 'prijem_z_prodeje' } ) \
#                 .round( 2 )
```
</details>


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.P16hnt14CM-mJ5eAKykxLAHaHa%26pid%3DApi&f=1&ipt=27e3f5b23fe44567ad0730192d5dcb13c7c23aa6e87579d16df05e27461dc12c&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

## Pivot tabulky

---


*Pivot tabulky* (*kontingenční tabulky*) jsou užitečné pro přehledné zobrazení a analýzu dat z tabulkových zdrojů.

Pomocí pivot tabulek můžeš **seskupit data podle určitých kategorií** a provést agregaci hodnot.

Nejprve si představ situaci bez pivot tabulek, pomocí ukázky níže:

In [67]:
import seaborn

<br>

V této ukazce použiješ vzorová data týkající se [nehody lodi Titanic](https://en.wikipedia.org/wiki/Sinking_of_the_Titanic):

In [68]:
df_titanic = seaborn.load_dataset('titanic')

In [69]:
df_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


<br>

Pro jakoukoliv pokročilou analýzu dat, potřebuješ údaj seskupit.

Tak jak jsi do prováděl doposud, pomocí `groupby`:

In [70]:
df_titanic["sex"].unique()

array(['male', 'female'], dtype=object)

In [71]:
df_titanic.groupby('sex')

<br>

Potřebuješ průměrnou hodnotu, ať je na sloupečku `survived` vidět poměr přeživších:

In [72]:
df_titanic.groupby('sex')['survived'].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

<br>

Pro lepší čitelnost **zaokrouhlím výsledek na dvě desetinné čísla**:

In [73]:
df_titanic.groupby('sex')['survived'].mean().round(2)

sex
female    0.74
male      0.19
Name: survived, dtype: float64

Takový průzkum z této studie ti dá jasný pohled na věc:
* 3 ze 4 žen přežily,
* 1 z 5 mužů přežil.

<br>

Pokud budeš potřebovat detailnější analýzy, budeš potřebovat více dat.

Třeba situaci, kde kromě pohlaví, bereš v potaz **třídu cestujících** `class`:

In [74]:
df_titanic["class"].unique()

['Third', 'First', 'Second']
Categories (3, object): ['First', 'Second', 'Third']

In [75]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean()

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

In [76]:
type(df_titanic.groupby(['sex', 'class'])[['survived']])

pandas.core.groupby.generic.DataFrameGroupBy

In [77]:
type(df_titanic.groupby(['sex', 'class'])['survived'].mean())

pandas.core.series.Series

In [78]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean().round(2)

survived
sex    class           
female First       0.97
       Second      0.92
       Third       0.50
male   First       0.37
       Second      0.16
       Third       0.14

<br>

Metodou `unstack` si můžeš vytvořit nové sloupečky, které jsou postavené na novém Indexu, nebo Indexech (*Multiindex*):

In [79]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


Takový průzkum ti dá skutečně lepší pohled na věc.

Současně ale roste **náročnost ohlášení**.

Zápis "bobtná" a **stává se náročnějším na přečtení a pochopení**.

<br>

### Pivot tabulka

---

Podobné řešení ti nabízí funkce `pivot_table`:

In [80]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


In [82]:
df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

sex,female,male
class,,
First,0.97,0.37
Second,0.92,0.16
Third,0.50,0.14


Zásádním rozdílem je ovšem **čitelnost**, kterou máš pro tuto variantu zápisu.

```
Clarity beats purity
```

Pomocí vhodných argumentů, můžeš doplnit vysvětlivky tam, kde funkce `groupby` nemohla.

Stejně platí, že pokud budeš potřebovat **další Index**, můžeš si pomoci funkcí `cut`:

In [83]:
df_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [84]:
df_titanic['age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64

In [85]:
from pandas import cut

In [86]:
age = cut(df_titanic['age'], [1, 25, 80])

In [87]:
age  # < xxx , xxx )

0       (1.0, 25.0]
1      (25.0, 80.0]
2      (25.0, 80.0]
3      (25.0, 80.0]
4      (25.0, 80.0]
           ...     
886    (25.0, 80.0]
887     (1.0, 25.0]
888             NaN
889    (25.0, 80.0]
890    (25.0, 80.0]
Name: age, Length: 891, dtype: category
Categories (2, interval[int64, right]): [(1, 25] < (25, 80]]

In [88]:
df_titanic.pivot_table('survived', index=['sex', age], columns='class').round(2)

class            First  Second  Third
sex    age                           
female (1, 25]    0.93    0.97   0.48
       (25, 80]   0.98    0.89   0.38
male   (1, 25]    0.46    0.16   0.14
       (25, 80]   0.38    0.08   0.15

Pomocí funkce `cut` doplníš tabulku na **MultiIndex**.

Ten nyní vytvoří i rozsah pro věkovou kategorii.

<br>

**🧠 CVIČENÍ 🧠, procvič si pivot tabulky**

Ze zadaného datasetu vytvoř **pivot tabulku**, která zobrazí **počet prodaných kusů ovoce** pro **každý obchod** a **druh ovoce**.

In [89]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
# pivot_table()

<details>
    <summary>▶️ Řešení</summary>
    
```python
pivot_tabulka = df_ovoce.pivot_table(values="prodane_mnozstvi", index="pobocka", columns="ovoce")
```
</details>

---